In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from lstm_v import *
from test_utils import *

# LSTM comparision

## Vanilla LSTM
<center>
<img src="assets/LSTMV.png" width="80%">
</center>

In [2]:
# compare nn.LSTM , hand coded LSTM -> Cells
lstm_tests(nn.LSTM,LSTM_Cell)

B=64 , T=200 , input dimension=128, hidden dimemnsion=256
[LSTM] Time taken: 0.171681 seconds
[recoded LSTM_Cell] Time taken: 0.096300 seconds
input :  torch.Size([64, 200, 128])
output :  torch.Size([64, 200, 256])
Shapes matched.


## minLSTM: A Minimal LSTM

<center>
<img src="assets/LSTMv2.png" width="80%" />
</center>

In [3]:
# compare nn.LSTM , MinLSTM -> Cells
lstm_tests(nn.LSTM,minLSTM_Cell)

B=64 , T=200 , input dimension=128, hidden dimemnsion=256
[LSTM] Time taken: 0.084651 seconds
[recoded minLSTM_Cell] Time taken: 0.051105 seconds
input :  torch.Size([64, 200, 128])
output :  torch.Size([64, 200, 256])
Shapes matched.


## Log-space minGRU

In [4]:
# compare nn.LSTM , log-space minLSTM -> Cells
lstm_tests(nn.LSTM,log_minLSTM_Cell)

B=64 , T=200 , input dimension=128, hidden dimemnsion=256
[LSTM] Time taken: 0.103004 seconds
[recoded log_minLSTM_Cell] Time taken: 0.078236 seconds
input :  torch.Size([64, 200, 128])
output :  torch.Size([64, 200, 256])
Shapes matched.


## Prallel minLSTM

In [12]:
B, T, input_dim, hidden_dim = 64,200, 128,256

# Random sequence input (batch_size, seq_len, input_dim)
x = torch.randn(B,T, input_dim)
# Initial hidden state (batch_size, hidden_dim)
h0 = torch.randn(B, hidden_dim)

parallel_minLSTM = parallel_log_minLSTM(input_dim,hidden_dim); parallel_minLSTM

parallel_log_minLSTM(
  (linear_f): Linear(in_features=128, out_features=256, bias=True)
  (linear_i): Linear(in_features=128, out_features=256, bias=True)
  (linear_h): Linear(in_features=128, out_features=256, bias=True)
)

In [13]:
parallel_minLSTM(x,h0).shape

torch.Size([64, 200, 256])

In [14]:
v_lstm = nn.LSTMCell(input_dim,hidden_dim); v_lstm

LSTMCell(128, 256)

In [16]:
h = h0
c0 = torch.randn(B, hidden_dim)
outputs = []
for t in range(T):
    h,c = v_lstm(x[:, t, :], (h,c0))
    outputs.append(h.unsqueeze(1))
out = torch.cat(outputs, dim=1)


In [17]:
out.shape == parallel_minLSTM(x,h0).shape

True